# Development for NeonPandas

In [1]:
import pandas as pd 
import neonpandas as npd

## Load Pets Dataset

In [2]:
pets = pd.read_csv('pets.csv')
pets

,name,species,color,age,behavior
0,Ralph,Dog,black,10.0,NaN
1,Pip,Cat,yellow,6.0,good
2,Babe,Pig,NaN,3.0,NaN
3,Bubbles,Fish,red,NaN,acceptable
4,Freckles,Horse,brown,NaN,NaN


## NeonPandas NodeFrame

In [3]:
pets = npd.NodeFrame(pets, id_col='name', lbl_col='species', labels={'Pet'})
pets

,labels,name,color,age,behavior
0,"{Dog, Pet}",Ralph,black,10.0,NaN
1,"{Cat, Pet}",Pip,yellow,6.0,good
2,"{Pet, Pig}",Babe,NaN,3.0,NaN
3,"{Fish, Pet}",Bubbles,red,NaN,acceptable
4,"{Horse, Pet}",Freckles,brown,NaN,NaN


## Set up Graph

In [5]:
graph = npd.Graph(uri='bolt://localhost:7687', auth=('neo4j', 'neonpandas'))

### Creates Nodes

In [ ]:
graph.create_nodes(pets)

### Create Constraints

In [8]:
constraints = npd.NodeFrame([
    {'labels': 'Pet', 'property': 'name'}, 
    {'labels': 'Owner', 'property': 'name'}
], lbl_col='labels')
constraints

,labels,property
0,{Pet},name
1,{Owner},name


In [ ]:
# create from dataframe
graph.create_node_constraints(constraints)

## NeonPandas EdgeFrame

In [10]:
edges = pd.DataFrame([
    {'src': 'Ralph', 'rel_type': 'IS_FRIENDLY_WITH', 'dest': 'Bubbles'},
    {'src': 'Pip', 'rel_type': 'IS_MEAN_TO', 'dest': 'Babe'},
    {'src': 'Ralph', 'rel_type': 'IS_MEAN_TO', 'dest': 'Freckles'},
    {'src': 'Freckles', 'rel_type': 'IS_FRIENDLY_WITH', 'dest': 'Babe'}
])
edges

,src,rel_type,dest
0,Ralph,IS_FRIENDLY_WITH,Bubbles
1,Pip,IS_MEAN_TO,Babe
2,Ralph,IS_MEAN_TO,Freckles
3,Freckles,IS_FRIENDLY_WITH,Babe


In [11]:
class EdgeFrame(pd.DataFrame):
    def __init__(self, data, src_col:str='src', rel_col:str='rel_type', dest_col:str='dest'):
        super().__init__(data)
        self.whatami = "NeonPandas EdgeFrame"
        self.src_col = src_col
        self.rel_col = rel_col
        self.dest_col = dest_col

In [12]:
test = EdgeFrame(edges)
test

,src,rel_type,dest
0,Ralph,IS_FRIENDLY_WITH,Bubbles
1,Pip,IS_MEAN_TO,Babe
2,Ralph,IS_MEAN_TO,Freckles
3,Freckles,IS_FRIENDLY_WITH,Babe


#### Requirements for EdgeFrame
- Designate source, relationship-type, and dest columns (i.e. src, dest, rel_type)
- Ability to join **src** and **dest** cols with NodeFrame to get ID info (e.g. labels, id_props, etc.)
- `create_edges()` method with option to set **src** and **dest** (individually) to `MATCH` or `MERGE`
- Assumes all columns in EdgeFrame beyond **src**, **dest**, and **rel-type** are edge properties

In [ ]:
query = """UNWIND $edges AS edge
        MATCH (s:Pet {name: edge.src})
        MATCH (d:Pet {name: edge.dest})
        WITH s,d,edge
        CALL apoc.merge.relationship(s, edge.rel_type, {}, {}, d) YIELD rel
        RETURN COUNT(rel)"""

In [ ]:
graph.create_relationships(edges, query)

### Node Joining
Perform _join_ operations with an input DataFrame against nodes in Neo4j.

In [18]:
new_pets = pd.DataFrame([
    {'name': 'Betsy', 'age': 2, 'species': 'Cow'},
    {'name': 'Carrie', 'species': 'Rabbit'}
])
new_pets = npd.NodeFrame(new_pets, id_col='name', labels={'Pet'}, lbl_col='species')
#all_pets = pd.concat([pets, new_pets], sort=False).reset_index(drop=True)
#all_pets

In [19]:
new_pets

,labels,name,age
0,"{Pet, Cow}",Betsy,2.0
1,"{Rabbit, Pet}",Carrie,NaN


In [21]:
graph.create_nodes(new_pets)

### Semi-Join
Check which nodes in DataFrame exist in Neo4j.

In [ ]:
graph.semi_join(all_pets, on='name', labels='Pet')

### Anti-Join
Check which nodes in DataFrame do not exist in Neo4j.

In [ ]:
graph.anti_join(all_pets, on='name', labels='Pet')

## Match Nodes
Search for nodes via `MATCH` statement

In [ ]:
graph.match_nodes(labels={'Pet'}, limit=3)

In [ ]:
## add properties to search
graph.match_nodes(labels={'Pet'}, properties={'name': 'Ralph'})

## DataFrame Object

In [ ]:
class Test(pd.DataFrame):
    def __init__(self, data, column:str=None, labels=None):
        super().__init__(data)
        self.whatami = "I am a NeonPandas DataFrame"
        self._set_labels(column=column, labels=labels)
        
    def _set_labels(self, column:str=None, labels:set=None) -> list:
        if column is not None and labels is None:
            assert column in self.columns
            _lbls = self[column].apply(lambda x: df_tools.conform_to_set(x))
        elif column is not None and labels is not None:
            _lbls = self[column].apply(lambda x: {x}.union(df_tools.conform_to_set(labels)))
        elif column is None and labels is not None:
            labels = df_tools.conform_to_set(labels)
            _lbls = [labels for i in range(len(self))]
        else:
            raise ValueError("Must provide either 'labels' or 'use_column' as input for attribute type.")
        # finish processing dataframe and labels column
        self.drop(columns=[column], inplace=True)
        # set labels as column
        self.insert(0, 'labels', _lbls)
        return

In [ ]:
pets_data = df_tools.convert_to_records(all_pets)
pets_data[0]

In [ ]:
pets_test = Test(pets_data, column='labels')
pets_test

In [ ]:
pets = pd.read_csv('pets.csv')
pets

In [ ]:
pets_test = Test(pets, column='species', labels={'Animal', 'Pet'})
pets_test